In [75]:
import pandas as pd
from pathlib import Path

month = 1
year = 2024

path = Path("..") / "data" / "raw" / f"rides_{year}_{month:02}.csv"

rides = pd.read_csv(path)
rides.head()

C:\Users\HP\AppData\Local\Temp\ipykernel_24836\785314117.py:9: DtypeWarning: Columns (5,7) have mixed types. Specify dtype option on import or set low_memory=False.
  rides = pd.read_csv(path)


,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
0,5078F3D302000BD2,electric_bike,2024-01-22 18:43:19.012,2024-01-22 18:48:10.708,Frederick Douglass Blvd & W 145 St,7954.12,St Nicholas Ave & W 126 St,7756.10,40.823072,-73.941738,40.811432,-73.951878,member
1,814337105D37302A,electric_bike,2024-01-11 19:19:18.721,2024-01-11 19:47:36.007,W 54 St & 6 Ave,6771.13,E 74 St & 1 Ave,6953.08,40.761822,-73.977036,40.768974,-73.954823,member
2,A33A920E2B10710C,electric_bike,2024-01-30 19:17:41.693,2024-01-30 19:32:49.857,E 11 St & Ave B,5659.11,W 10 St & Washington St,5847.06,40.727592,-73.979751,40.733424,-74.008515,casual
3,A3A5FC0DD7D34D74,electric_bike,2024-01-27 11:27:01.759,2024-01-27 11:38:01.213,W 54 St & 6 Ave,6771.13,E 74 St & 1 Ave,6953.08,40.761779,-73.977144,40.768974,-73.954823,member
4,6F96728ECEFBDAA4,electric_bike,2024-01-16 15:15:41.000,2024-01-16 15:29:26.156,Madison Ave & E 99 St,7443.01,E 74 St & 1 Ave,6953.08,40.789808,-73.952214,40.768974,-73.954823,member


In [62]:
rides.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1888085 entries, 0 to 1888084
Data columns (total 13 columns):
 #   Column              Dtype  
---  ------              -----  
 0   ride_id             object 
 1   rideable_type       object 
 2   started_at          object 
 3   ended_at            object 
 4   start_station_name  object 
 5   start_station_id    object 
 6   end_station_name    object 
 7   end_station_id      object 
 8   start_lat           float64
 9   start_lng           float64
 10  end_lat             float64
 11  end_lng             float64
 12  member_casual       object 
dtypes: float64(4), object(9)
memory usage: 187.3+ MB


In [76]:
rides_cp = rides.copy()

rides_cp.isnull().sum()

ride_id                  0
rideable_type            0
started_at               0
ended_at                 0
start_station_name    1160
start_station_id      1160
end_station_name      5505
end_station_id        5505
start_lat                0
start_lng                0
end_lat                551
end_lng                551
member_casual            0
dtype: int64

In [64]:
filter_not_null = rides_cp.notnull().all(axis=1)

In [65]:
rides_cp.dropna(inplace=True)

In [66]:
rides_cp["started_at"] = pd.to_datetime(rides_cp["started_at"], errors='coerce')
rides_cp["ended_at"] = pd.to_datetime(rides_cp["ended_at"], errors='coerce')

# Now safely subtract datetimes
rides_cp["duration"] = rides_cp["ended_at"] - rides_cp["started_at"]

rides_cp.head()

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,duration
0,5078F3D302000BD2,electric_bike,2024-01-22 18:43:19.012,2024-01-22 18:48:10.708,Frederick Douglass Blvd & W 145 St,7954.12,St Nicholas Ave & W 126 St,7756.10,40.823072,-73.941738,40.811432,-73.951878,member,0 days 00:04:51.696000
1,814337105D37302A,electric_bike,2024-01-11 19:19:18.721,2024-01-11 19:47:36.007,W 54 St & 6 Ave,6771.13,E 74 St & 1 Ave,6953.08,40.761822,-73.977036,40.768974,-73.954823,member,0 days 00:28:17.286000
2,A33A920E2B10710C,electric_bike,2024-01-30 19:17:41.693,2024-01-30 19:32:49.857,E 11 St & Ave B,5659.11,W 10 St & Washington St,5847.06,40.727592,-73.979751,40.733424,-74.008515,casual,0 days 00:15:08.164000
3,A3A5FC0DD7D34D74,electric_bike,2024-01-27 11:27:01.759,2024-01-27 11:38:01.213,W 54 St & 6 Ave,6771.13,E 74 St & 1 Ave,6953.08,40.761779,-73.977144,40.768974,-73.954823,member,0 days 00:10:59.454000
4,6F96728ECEFBDAA4,electric_bike,2024-01-16 15:15:41.000,2024-01-16 15:29:26.156,Madison Ave & E 99 St,7443.01,E 74 St & 1 Ave,6953.08,40.789808,-73.952214,40.768974,-73.954823,member,0 days 00:13:45.156000


In [67]:
# Here timedelta is an object in datetime module which makes it easy to perform arthimetic operations on the timestamps including days, hours, 
# minutes etc

print(rides_cp["duration"].quantile(0))

print(rides_cp["duration"].quantile(0.01))

print(rides_cp["duration"].quantile(0.99))

print(rides_cp["duration"].quantile(0.999))


0 days 00:01:00.044000
0 days 00:01:20.040760
0 days 00:46:16.484479999
0 days 02:24:25.731416


In [68]:
rides_cp["duration"].describe()

count                      1881977
mean     0 days 00:10:50.949568386
std      0 days 00:20:47.307345380
min         0 days 00:01:00.044000
25%         0 days 00:04:39.642000
50%         0 days 00:07:43.293000
75%         0 days 00:12:56.011000
max         1 days 00:59:06.090000
Name: duration, dtype: object

In [69]:
duration_filter = (rides_cp["duration"] > pd.Timedelta(0) ) & (rides_cp["duration"] <=pd.Timedelta(hours = 5) )

sum(~duration_filter) # This will tell how many instances we are gonna remove

1013

In [70]:
sorted_df = rides_cp.sort_values(by="started_at", ascending=True)  

# Get the top 10 (smallest) and bottom 10 (largest) values  
top_10 = sorted_df.head(10)  
bottom_10 = sorted_df.tail(10)  

top_10


,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,duration
1107210,BD613F756C39C4D0,classic_bike,2023-12-31 13:50:28.976,2024-01-01 08:41:20.899,N 12 St & Bedford Ave,5450.04,S 4 St & Rodney St,5156.05,40.720798,-73.954847,40.709340,-73.956080,casual,0 days 18:50:51.923000
1077054,4B080070FD50B858,electric_bike,2023-12-31 14:55:38.927,2024-01-01 07:27:09.362,W 37 St & 5 Ave,6398.06,Amsterdam Ave & W 79 St,7311.02,40.750380,-73.983390,40.782939,-73.978652,member,0 days 16:31:30.435000
211345,24F86164747D9B56,classic_bike,2023-12-31 14:57:39.538,2024-01-01 14:13:10.446,Madison St & Montgomery St,5262.09,Allen St & Stanton St,5484.09,40.713126,-73.984844,40.722055,-73.989111,casual,0 days 23:15:30.908000
656419,165867F584717117,classic_bike,2023-12-31 15:57:36.402,2024-01-01 14:03:00.685,Cadman Plaza E & Red Cross Pl,4821.06,St Marks Pl & 4 Ave,4249.10,40.699918,-73.989718,40.681778,-73.979890,member,0 days 22:05:24.283000
841812,7FA2408E227974A4,classic_bike,2023-12-31 17:32:19.418,2024-01-01 07:02:54.951,Central Park S & 6 Ave,6876.04,Central Park West & W 68 St,7079.06,40.765909,-73.976342,40.773407,-73.977825,casual,0 days 13:30:35.533000
1481924,7B5A7F2EBE5619B7,classic_bike,2023-12-31 17:54:00.651,2024-01-01 16:39:31.783,6 Ave & W 34 St,6364.1,Bleecker St & Crosby St,5679.08,40.749640,-73.988050,40.726156,-73.995102,member,0 days 22:45:31.132000
1178677,D577198D5B511ADB,classic_bike,2023-12-31 18:49:42.512,2024-01-01 12:01:59.014,14 St & 7 Ave,3731.11,Underhill Ave & Lincoln Pl,4042.08,40.663779,-73.983968,40.674012,-73.967146,member,0 days 17:12:16.502000
1178771,27DB996CA83D276C,classic_bike,2023-12-31 18:49:42.620,2024-01-01 12:01:59.014,14 St & 7 Ave,3731.11,Underhill Ave & Lincoln Pl,4042.08,40.663779,-73.983968,40.674012,-73.967146,member,0 days 17:12:16.394000
1801088,147EB043317659FB,classic_bike,2023-12-31 19:22:31.619,2024-01-01 12:19:24.003,Metropolitan Ave & 65 Ln,5160.01,E Fordham Rd & Webster Ave,8582.09,40.712250,-73.891110,40.861748,-73.891050,casual,0 days 16:56:52.384000
217409,DAB454B0ECFEC7B7,classic_bike,2023-12-31 19:34:33.299,2024-01-01 02:22:22.488,Broadway & W 133 St,7903.02,Broadway & Moylan Pl,7823.03,40.819034,-73.956156,40.814326,-73.959025,casual,0 days 06:47:49.189000


In [71]:
bottom_10

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,duration
1184295,DBD5F1EDA05272A2,classic_bike,2024-01-31 23:55:15.155,2024-01-31 23:59:37.157,Carmine St & 6 Ave,5763.03,W Broadway & Spring St,5569.06,40.730386,-74.002150,40.724947,-74.001659,member,0 days 00:04:22.002000
587501,CF4D2CE8EF506FDA,electric_bike,2024-01-31 23:55:15.552,2024-01-31 23:56:42.709,E 11 St & Ave B,5659.11,Ave A & E 14 St,5779.11,40.727697,-73.979802,40.730311,-73.980472,member,0 days 00:01:27.157000
986030,7C068CC129D59029,classic_bike,2024-01-31 23:55:33.484,2024-01-31 23:58:13.282,E 13 St & 2 Ave,5820.08,St Marks Pl & 1 Ave,5626.13,40.731539,-73.985302,40.727791,-73.985649,member,0 days 00:02:39.798000
335379,1C08197BC3A120F6,electric_bike,2024-01-31 23:55:43.784,2024-01-31 23:59:29.515,20 Ave & 38 St,7196.04,31 St & Ditmars Blvd,7144.02,40.777268,-73.902671,40.776168,-73.910485,casual,0 days 00:03:45.731000
1312377,E8E94F7C1E0991F8,electric_bike,2024-01-31 23:55:48.332,2024-01-31 23:58:16.455,Eastern Pkwy & Troy Ave,3862.07,Kingston Ave & Carroll St,3831.03,40.669338,-73.936926,40.666880,-73.942500,member,0 days 00:02:28.123000
152866,8372FE21DCD75179,classic_bike,2024-01-31 23:55:59.368,2024-01-31 23:57:27.770,Kent Ave & N 7 St,5489.03,N 11 St & Kent Ave,5489.04,40.720368,-73.961651,40.722482,-73.959219,member,0 days 00:01:28.402000
1183902,FF5497A5A844C851,electric_bike,2024-01-31 23:56:38.776,2024-01-31 23:59:28.329,Sherman Ave & Thayer St,8583.02,W 190 St & Broadway,8474.02,40.863133,-73.926973,40.856487,-73.932970,member,0 days 00:02:49.553000
291989,00DB0ACCAA339429,classic_bike,2024-01-31 23:57:04.310,2024-01-31 23:59:47.214,Adam Clayton Powell Blvd & W 141 St,7893.05,W 147 St & Adam Clayton Powell Blvd,7971.07,40.819241,-73.941057,40.822810,-73.937413,member,0 days 00:02:42.904000
1041975,335E4CF575E8F41A,electric_bike,2024-01-31 23:57:04.650,2024-01-31 23:59:05.024,W 100 St & Broadway,7580.01,West End Ave & W 94 St,7524.09,40.797373,-73.970256,40.794165,-73.974124,member,0 days 00:02:00.374000
315390,6F464FCB160C98DF,classic_bike,2024-01-31 23:58:30.270,2024-01-31 23:59:36.654,Forsyth St & Broome St,5453.05,Delancey St & Eldridge St,5414.07,40.718939,-73.992663,40.719383,-73.991479,member,0 days 00:01:06.384000


In [72]:
filter_date_range = (rides_cp["started_at"]>= "2024-01-01") & (rides_cp["started_at"] < "2024-02-01")
sum(~filter_date_range)

364

In [73]:
final_filter = duration_filter & filter_date_range & filter_not_null

numbers_dropped = final_filter.shape[0] - sum(final_filter) # numbers dropped
numbers_dropped
numbers_dropped/final_filter.shape[0] * 100

0.3952152577876526

In [74]:
rides = rides[final_filter]

rides = rides[["started_at", "start_station_id"]]

rides.rename(columns={
    "started_at": "pickup_datetime",
    "start_station_id": "pickup_location_id"
}, inplace=True)

rides.head()

year = 2024

month = 1

path = Path("..") / "data" / "processed" / f"rides_{year}_{month :02}.csv"

rides.to_csv(path,index = False)